In [196]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Objective: To understand how turnover affects profit in Las Vegas from 2015 - 2022

In [177]:
data = pd.read_csv("data.csv")

In [178]:
data.head()

,Unnamed: 0,Type,Date,Num,Name,Memo,Clr,Split,Amount,Balance
0,CHASE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Check,12/17/14,NaN,FRANCHISE TAX BOARD,2014 chase chk #444,X,LLC Tax,-800.0,-800.0
2,NaN,Deposit,12/30/14,NaN,"GoldenWest Management, Inc",1828 BRIGHT LEAF 2014,X,Rental Income,12540.0,11740.0
3,NaN,Check,12/30/14,NaN,"GoldenWest Management, Inc",BRIGHT LEAF 2014,X,Management Fees,-900.0,10840.0
4,NaN,Check,12/30/14,NaN,State Farm Fire and Casualty Company,BRIGHT LEAF 2014,X,Insurance Expense,-532.0,10308.0


In [179]:
data["Date"] = pd.to_datetime(data["Date"])

In [180]:
data["Year"] = (data["Date"].apply(lambda x: x.year))

In [181]:
df = data.loc[data["Year"].isin(np.arange(2015, 2021+1))]
df = df.loc[df["Type"].isin(["Deposit", "Check"])]

In [182]:
df["is_bright_leaf"] = df["Memo"].str.lower().str.contains("bright|leaf")

In [183]:
df["is_fort_pike"] = df["Memo"].str.lower().str.contains("fort|pike")

In [184]:
def selector(row):
    if row["is_bright_leaf"]:
        return "bright leaf"
    elif row["is_fort_pike"]:
        return "fort pike"
    else:
        return None

In [185]:
df["Property"] = df.apply(selector, axis=1)

In [186]:
df

,Unnamed: 0,Type,Date,Num,Name,Memo,Clr,Split,Amount,Balance,Year,is_bright_leaf,is_fort_pike,Property
13,NaN,Deposit,2015-01-01,NaN,"GoldenWest Management, Inc",1/2015 Bright Leaf,X,Rental Income,1050.0,6157.03,2015.0,True,False,bright leaf
14,NaN,Check,2015-01-01,NaN,"GoldenWest Management, Inc",1/2015 Bright Leaf,X,Management Fees,-75.0,6082.03,2015.0,True,False,bright leaf
15,NaN,Deposit,2015-01-01,NaN,"GoldenWest Management, Inc",1/2015 Fort Pike,X,Rental Income,1550.0,7632.03,2015.0,False,True,fort pike
16,NaN,Check,2015-01-01,NaN,"GoldenWest Management, Inc",1/2015 Fort Pike,X,Management Fees,-100.0,7532.03,2015.0,False,True,fort pike
17,NaN,Check,2015-01-05,NaN,State Farm Fire and Casualty Company,1828 Bright Leaf 1/28/2015 to 1/28/2016,X,Insurance Expense,-536.0,6996.03,2015.0,True,False,bright leaf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,NaN,Check,2021-12-03,NaN,"GoldenWest Management, Inc",Zillow Group Ad Fee Fort Pike,X,Management Fees,-12.0,11000.63,2021.0,False,True,fort pike
591,NaN,Check,2021-12-12,NaN,ELDORADO THIRD CA,chase chk #593 Bright Leaf,X,Community Association Fee,-69.0,10931.63,2021.0,True,False,bright leaf
592,NaN,Check,2021-12-14,NaN,State Farm Fire and Casualty Company,1828 bright leaf,X,Insurance Expense,-613.0,10318.63,2021.0,True,False,bright leaf
593,NaN,Check,2021-12-27,NaN,State Farm Fire and Casualty Company,10001 fort pike,X,Insurance Expense,-783.0,9535.63,2021.0,False,True,fort pike


In [187]:
df.drop(["Unnamed: 0", "Date", "Num", "Clr", "Balance", "Memo", "is_bright_leaf", "is_fort_pike"], axis=1, inplace=True)

In [188]:
df

,Type,Name,Split,Amount,Year,Property
13,Deposit,"GoldenWest Management, Inc",Rental Income,1050.0,2015.0,bright leaf
14,Check,"GoldenWest Management, Inc",Management Fees,-75.0,2015.0,bright leaf
15,Deposit,"GoldenWest Management, Inc",Rental Income,1550.0,2015.0,fort pike
16,Check,"GoldenWest Management, Inc",Management Fees,-100.0,2015.0,fort pike
17,Check,State Farm Fire and Casualty Company,Insurance Expense,-536.0,2015.0,bright leaf
...,...,...,...,...,...,...
590,Check,"GoldenWest Management, Inc",Management Fees,-12.0,2021.0,fort pike
591,Check,ELDORADO THIRD CA,Community Association Fee,-69.0,2021.0,bright leaf
592,Check,State Farm Fire and Casualty Company,Insurance Expense,-613.0,2021.0,bright leaf
593,Check,State Farm Fire and Casualty Company,Insurance Expense,-783.0,2021.0,fort pike


### Comparing revenue of each property from 2015-2021

In [189]:
income = df.loc[df["Split"]=="Rental Income"].groupby(["Property", "Year"]).sum().reset_index()

In [190]:
income

,Property,Year,Amount
0,bright leaf,2015.0,12800.00
1,bright leaf,2016.0,12975.00
2,bright leaf,2017.0,12595.00
3,bright leaf,2018.0,13140.00
4,bright leaf,2019.0,11280.00
5,bright leaf,2020.0,14304.20
6,bright leaf,2021.0,14940.00
7,fort pike,2015.0,18910.00
8,fort pike,2016.0,18545.00
9,fort pike,2017.0,18705.00


In [191]:
fig = px.line(income, x="Year", y="Amount", color="Property")

In [192]:
fig

### Looking at Repairs and Maintenance as a proportion of Revenue

In [207]:
#Fort Pike
fp = df.loc[df["Property"]=="fort pike"]

fp_rev = fp.loc[df["Split"]=="Rental Income"]
rev = fp_rev.groupby("Year").sum().reset_index()

fp_rm = fp.loc[df["Split"]=="Repairs and Maintenance"]
rm = fp_rm.groupby("Year").sum().reset_index()
rm["Amount"]= rm["Amount"]*-1
rm

,Year,Amount
0,2015.0,360.00
1,2016.0,704.00
2,2017.0,240.00
3,2018.0,1550.50
4,2019.0,200.00
5,2020.0,3029.27
6,2021.0,4776.70


In [213]:
trace1 = go.Bar(x = rev["Year"], y=rev["Amount"], marker=dict(color="green", opacity=1), name="Revenue")
trace2 = go.Bar(x = rm["Year"], y=rm["Amount"], marker=dict(color="red", opacity=1), name="Repairs and Maintenance")
data=[trace1, trace2]
layout = go.Layout(title = "Repairs and Maintenace as a Proportion of Revenue", barmode="overlay", 
                   xaxis=dict(title="Year"), yaxis=dict(title= "Amount"))
fig = go.Figure(data,layout)
fig.show()

In [209]:
trace1

Bar({
    'marker': {'color': 'green', 'opacity': 0.05},
    'name': 'Revenue',
    'x': array([2015., 2016., 2017., 2018., 2019., 2020., 2021.]),
    'y': array([18910.  , 18545.  , 18705.  , 16770.  , 20530.  , 21115.  , 20662.16])
})

In [ ]:
Split = ["Rental Income", "Management Fees", "Community Association Fee", "Insurance Expense", "Repairs and Maintenace", "LLC Tax", "Utilities"]

In [ ]:
If looking at community association fee for Fort Pike look at "MOUNTAINS EDGE MASTER ASSOCIATION"

Clark county water reclamation is fort pike